In [1]:
from src.data_management.dataset import CxrDatasetVer1, DatasetSplit
ds = CxrDatasetVer1(
    metadata_path='data/metadata/mimic-cxr-2.0.0-metadata.csv',
    split_path='data/metadata/mimic-cxr-2.0.0-split.csv',
    report_dir='data/mimic-cxr-reports',
    img_dir='data/cxr_dataset',
    split=DatasetSplit.TRAIN
)
img, text = ds.__getitem__(0)

INFO:src.data_management.dataset:__getitem__ executed in 0.05 s
